In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
from os import chdir 
chdir("../")

In [2]:
df = pd.read_csv("data/aggregated.csv")

In [18]:
from scripts.preprocessing import drop_values, encode_cats, add_busy_feature
cleaned_df = drop_values(df)
cleaned_df = encode_cats(cleaned_df)
cleaned_df = cleaned_df.dropna()
cleaned_df = add_busy_feature(cleaned_df, 2727.75748502994+400) 
cleaned_df = cleaned_df.drop(columns=['FL_DATE'])
target = cleaned_df['ARR_DEL15']
cleaned_df = cleaned_df.drop(columns=['ARR_DEL15'])

In [19]:
from scripts.model_helpers import get_standard_df
## Normalize
standard = get_standard_df(cleaned_df)


In [20]:
from sklearn.decomposition import PCA

for i in range(5, len(cleaned_df.columns)):
    pca = PCA(n_components=i)
    principalComponents = pca.fit_transform(standard)
    print(i, pca.explained_variance_ratio_)
    print(i, sum(pca.explained_variance_ratio_))

5 [0.25445474 0.13933572 0.1302438  0.12587565 0.12405129]
5 0.7739611978685312
6 [0.25445474 0.13933572 0.1302438  0.12587565 0.12405129 0.1172671 ]
6 0.8912282996769264
7 [0.25445474 0.13933572 0.1302438  0.12587565 0.12405129 0.1172671
 0.10700568]
7 0.9982339794654919
8 [0.25445474 0.13933572 0.1302438  0.12587565 0.12405129 0.1172671
 0.10700568 0.00176602]
8 1.0000000000000002


### Best PCA value appears to be 7 since it holds 99% of the total data meaning

In [33]:
pca = PCA(n_components=7)
principalComponents = pca.fit_transform(cleaned_df)
principalDf = pd.DataFrame(data = principalComponents, columns = [f'principal component {e}' for e in range(0, 7)])
principalDf["ARR_DEL15"] = [e for e in target]

In [42]:
from sklearn.ensemble import RandomForestClassifier
from scripts.model_helpers import split_train_predict
subset = principalDf.sample(20000)
pred, y_test = split_train_predict(subset, RandomForestClassifier())

In [43]:
from sklearn.metrics import accuracy_score
print(accuracy_score(pred, y_test))

0.8136


## Conclusions 
### PCA provided an accuracy score that's less than our previous baseline of .82 when we introduced the BUSY_TIME column